<a id="top"></a>
# Masking Persistence in WFC3/IR Images
***
## Learning Goals

This notebook shows how to use the Hubble Space Telescope WFC3/IR persistence model to flag pixels affected by persistence in the calibrated (FLT) science images. When the images are sufficiently dithered to step over the observed persistence artifacts, AstroDrizzle may be used to exclude those flagged pixels when combining the FLT frames. 

By the end of this tutorial, you will:

- Download images and persistence products from MAST.
- Flag affected pixels in the data quality arrays of the FLT images.
- Redrizzle the FLT images to produce a "clean" DRZ combined product.

## Table of Contents

[Introduction](#intro) <br>
[1. Imports](#import) <br>
[2. Data](#data) <br>
- [2.1 Download the WFC3/IR observations from MAST](#images) <br>
- [2.2 Download the persistence model products](#model) <br>

[3. Analysis](#analysis) <br>
- [3.1 Display the images](#display) <br>
- [3.2 Use the persistence model to add DQ flags](#flag) <br>
- [3.3 Redrizzle the FLT data and apply the new DQ flags](#drizzle) <br>
- [3.4 Compare the original and corrected DRZ files](#compare) <br>
- [3.5 Compare the original and corrected WHT files](#weight) <br>

[4. Conclusions](#conclusions) <br>
[Additional Resources](#add) <br>
[About this Notebook](#about) <br>
[Citations](#cite) <br>

## Introduction <a id="intro"></a>

Image persistence in the IR array occurs whenever a pixel is exposed to light that exceeds more than about half of the full well of a pixel in the array. Persistence can occur within a single visit, as the different exposures in a visit are dithered. Persistence also occurs from observations in a previous visit of completely different fields.

Image persistence is seen in a small, but non-negligible fraction of WFC3/IR exposures. Its properties are discussed in in [Section 5.7.9](https://hst-docs.stsci.edu/wfc3ihb/chapter-5-wfc3-detector-characteristics-and-performance/5-7-ir-detector-characteristics-and-performance#id-5.7IRDetectorCharacteristicsandPerformance-5.7.95.7.9ImagePersistence) of the WFC3 Instrument Handbook and in [Chapter 8](https://hst-docs.stsci.edu/wfc3dhb/chapter-8-persistence-in-wfc3-ir) of the WFC3 Data Handbook. Persistence is primarily a function of the degree to which a pixel is filled (in electrons) and the time since this occurred. Additional information is available from the [WFC3 Persistence Webpage](https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/ir-persistence).

As described in [Section 8.3](https://hst-docs.stsci.edu/wfc3dhb/chapter-8-persistence-in-wfc3-ir/8-3-mitigating-the-effects-of-persistence) of the Data Handbook, there are two possible ways to mitigate persistence: 1.) exclude the affected pixels from the analysis or 2.) subtract the persistence model directly from the image.

This notebook illustrates the first method and shows how to use model to flag affected pixels in the data quality (DQ) array of each FLT image. When the images are sufficiently dithered, affected regions of the detector may be replaced with 'good' pixels from other exposures in the visit when combining the exposures with AstroDrizzle. Note that this reduces the effective exposure time in those regions of the combined image. 

In the second method, the persistence-corrected FLT frames, downloaded in [Section 2.2](#model) of this notebook, may be used directly for analysis or a scaled version of the persistence model may be subtracted from each FLT image until an adequate correction is achieved. In this case, flagging the affected pixels in the DQ arrays would not be required.

## 1. Imports <a id="import"></a>
    
This notebook assumes you have created the virtual environment in [WFC3 Library's](https://github.com/spacetelescope/WFC3Library) installation instructions.

We import: 

- *os* for setting environment variables
- *glob* for finding lists of files
- *shutil* for managing directories
- *numpy* for handling array functions
- *matplotlib.pyplot* for plotting data
- *astropy.io fits* for accessing FITS files

- *ccdproc* for querying keyword values in the FITS headers
- *astroquery* for downloading data from MAST
- *urllib* for obtaining the Persistence products from MAST
- *drizzlepac astrodrizzle* for combining images

In [ ]:
import os 
import glob 
import shutil

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

from ccdproc import ImageFileCollection
from astroquery.mast import Observations

import urllib
from drizzlepac import astrodrizzle

## 2. Data <a id="data"></a>

### 2.1 Download the WFC3/IR observations from MAST <a id="images"></a>

Here, we obtain WFC3/IR observations from GLASS program 13459, Visit 29 in the F140W filter. These exposures were impacted by persistence from grism G102 exposures obtained just prior to these. 

The following commands query MAST and then download the FLT and DRZ data products to the current directory.

In [ ]:
data_list = Observations.query_criteria(obs_id='ica529*',filters='F140W')

Observations.download_products(data_list['obsid'],mrp_only=False,download_dir='./data',
                               productSubGroupDescription=['FLT','DRZ'])

science_files = glob.glob('data/mastDownload/HST/*/*fits')

for im in science_files:
    root = im.split('/')[-1]
    os.rename(im,'./'+root)
shutil.rmtree('data/')

This Visit contains 4 consecutive dithered FLT exposures in the F140W filter, which are obtained in a single orbit. The following commands print the values of keywords describing those data, where the POSTARG* values represent the commanded x-axis and y-axis offsets in arcseconds. 

In [ ]:
collec = ImageFileCollection('./',
                             keywords=["asn_id","targname","filter","samp_seq","nsamp","exptime",
                             "postarg1","postarg2","date-obs","time-obs",], glob_include="ica529*flt.fits", ext=0)
out_table = collec.summary
out_table

### 2.2 Download the Persistence Model Products  <a id="model"></a>

To find the URL of the tar file containing the persistence fits files, visit the [PERSIST Search Form](https://archive.stsci.edu/prepds/persist/search.php) and search for dataset='ica529*'.

You may hover over the link in the Visit column to get the URL to the gzipped tar file for Visit 29. This URL is called in the Python code below. The persistence model data products we will use to create a mask are named 'rootname_persist.fits' and contain any contributions from either external or internal persistence. 

External persistence is defined as residual signal that is generated by an earlier visit, and internal persistence as that generated within the same visit as the image in question. External persistence typically comes from a prior scheduled WFC3/IR program and is not within the control of the observer. Internal persistence can be mitigated by the observer by dithering the exposures within a given visit. 

This cell may take several minutes to complete, as it will download persistence products for all images in the visit, and not just those for the F140W filter. 

In [ ]:
request=urllib.request.urlopen('https://archive.stsci.edu/pub/wfc3_persist/13459/Visit29/13459.Visit29.tar.gz')

tar=request.read()
with open('13459.Visit29.tar.gz', "wb") as gzfile:
    gzfile.write(tar)

!tar -zxvf 13459.Visit29.tar.gz

os.remove('13459.Visit29.tar.gz')

This creates a subdirectory named 13459.Visit29/ in the working directory with the following products for each calibrated 'flt.fits' image:
  * persist.fits:  The persistence model, including both internal and external persistence
  * extper.fits:   The persistence model, including only external persistence
  * flt_corr.fits: The corrected FLT image, equal to the difference between the image and the model ('flt.fits' - 'persist.fits').
  
Note that limitations in the accuracy of the model can result in corrected FLT images in which persistence is not properly removed. This is especially true when prior observations were obtained in scanning mode, where the model significantly underestimates the level of persistence. In this case the model may be iteratively scaled and subtracted from the FLT frame until the residual signal is fully removed.  

Alternately, this notebook shows how the user can flag the impacted pixels in the FLT data quality array then reprocess with AstroDrizzle to improve the combined image.

## 3. Analysis <a id="analysis"></a>

### 3.1 Display the images  <a id="display"></a>

The DRZ product combines the 4 individual FLT exposures and shows faint residual persistence from grism observations obtained just prior to these data.  An example of the 'Persistence Removal Evaluation' for the first dataset ica529rmq may be found [here](https://archive.stsci.edu/pub/wfc3_persist/13459/Visit29/ica529rmq_persist.html).

The DRZ pipeline product is shown on the left, and we can see that the persistence was partially, but not completely, removed by the cosmic-ray rejection functionality in AstroDrizzle. The persistence model for the first FLT dataset is shown on the right.  A pair of red boxes are overplotted on each image to highlight regions of the detector with the most visible grism persistence in the combined DRZ image. 

In [ ]:
drz =fits.getdata('ica529030_drz.fits',ext=1)
per1=fits.getdata('13459.Visit29/ica529rsq_persist.fits',ext=1)

fig=plt.figure(figsize=(20,8))
ax1=fig.add_subplot(1,2,1)
ax2=fig.add_subplot(1,2,2)

ax1.imshow(drz,  vmin=0.85,vmax=1.4,cmap='Greys_r',origin='lower')
ax2.imshow(per1, vmin=0.0,vmax=0.005,cmap='Greys_r',origin='lower')

ax1.set_title('ica529030_drz.fits (Drizzled Product = 4 FLTs)',fontsize=20)
ax2.set_title('ica529rsq_persist.fits (Model for a single FLT)',fontsize=20)

ax1.plot([0,280,280,0,0],[250,250,350,350,250],c='red')
ax2.plot([0,280,280,0,0],[250,250,350,350,250],c='red')
ax1.plot([200,480,480,200,200],[360,360,460,460,360],c='red')
ax2.plot([200,480,480,200,200],[360,360,460,460,360],c='red')

### 3.2 Use the persistence model to add DQ flags  <a id="flag"></a>

For any pixels in the model with a signal greater than 0.005 e-/sec, we add a flag of 16384 to the current DQ array values in each FLT frame. This threshold is flexible and should be determined empirically by the user based on the science objective and the fraction of pixels impacted. Note that the IR dark rate is 0.049 e-/s, so a threshold of 0.05 e-/s or 0.01 e-/s may be a more reasonable starting value in order to avoid flagging more pixels than can be filled in with the associated dithered FLT frames. 

In [ ]:
flt1_dq=fits.open('ica529rmq_flt.fits',mode='update')
per1   =fits.getdata('13459.Visit29/ica529rmq_persist.fits',ext=1)

flt2_dq=fits.open('ica529rsq_flt.fits',mode='update')
per2   =fits.getdata('13459.Visit29/ica529rsq_persist.fits',ext=1)

flt3_dq=fits.open('ica529s0q_flt.fits',mode='update')
per3   =fits.getdata('13459.Visit29/ica529s0q_persist.fits',ext=1)

flt4_dq=fits.open('ica529s6q_flt.fits',mode='update')
per4   =fits.getdata('13459.Visit29/ica529s6q_persist.fits',ext=1)

flt1_dq[3].data[per1 > 0.005] += 16384
flt2_dq[3].data[per2 > 0.005] += 16384
flt3_dq[3].data[per3 > 0.005] += 16384
flt4_dq[3].data[per4 > 0.005] += 16384

flt1_dq.close()
flt2_dq.close()
flt3_dq.close()
flt4_dq.close()

### 3.3 Redrizzle the FLT data and apply the new DQ flags  <a id="drizzle"></a>

Now, we recombine the FLT data with AstroDrizzle using the updated DQ arrays and compare with the pipeline DRZ data products. The following cell uses the default parameter values recommended for the IR detector, where `final_bits` tells AstroDrizzle which DQ flags to ignore (e.g. to treat as good data). All other flagged (non-zero) values will be treated as bad pixels and excluded from the combined image. 

In [ ]:
astrodrizzle.AstroDrizzle('ica529*flt.fits',output='ica529030_pcorr',
                          preserve=False, build=True, clean=True, 
                          skymethod='match', sky_bits='16',
                          driz_sep_bits='512,16', combine_type='median',
                          driz_cr_snr='5.0 4.0', driz_cr_scale='3.0 2.4', 
                          final_bits='512,16', num_cores=1)

### 3.4 Compare the original and corrected DRZ files  <a id="compare"></a>

Here, we display the drizzled image from the pipeline and the reprocessed drizzled image with persistence masked. 

In [ ]:
drz      =fits.getdata('ica529030_drz.fits',ext=1)
drz_corr =fits.getdata('ica529030_pcorr_drz.fits',ext=1)

fig=plt.figure(figsize=(20,8))
ax1=fig.add_subplot(1,2,1)
ax2=fig.add_subplot(1,2,2)

ax1.imshow(drz,      vmin=0.85,vmax=1.4,cmap='Greys_r',origin='lower')
ax2.imshow(drz_corr, vmin=0.85,vmax=1.4,cmap='Greys_r',origin='lower')

ax1.set_title('ica529030_drz.fits (Pipeline SCI)',fontsize=20)
ax2.set_title('ica529030_pcorr_drz.fits (Corrected SCI)',fontsize=20)

ax1.plot([0,280,280,0,0],[250,250,350,350,250],c='red')
ax2.plot([0,280,280,0,0],[250,250,350,350,250],c='red')
ax1.plot([200,480,480,200,200],[360,360,460,460,360],c='red')
ax2.plot([200,480,480,200,200],[360,360,460,460,360],c='red')

### 3.5 Compare the original and corrected WHT files  <a id="weight"></a>

When `final_wht_type='EXP'`, the drizzled WHT images provide an effective exposure time map of the combined array. 

In the plots below, we can see that the pipeline products have lower weight in the region impacted by the grism, but that only a single frame was flagged and rejected by the cosmic ray algorithm. The grey horizontal bars have a weight ~500 seconds compared to the total exposure ~700 seconds.

In the redrizzled 'corrected' WHT image, only a single exposure contributed to the darkest horizontal bars which have a value of ~200 seconds. The adjacent grey bars have a weight ~500 seconds and the rest of the WHT image is ~700 seconds. Users will need to experiment with the persistence masking thresholds and consider the size of the dithers in their individual datasets to determine the best masking strategy.

In [ ]:
wht      =fits.getdata('ica529030_drz.fits',ext=2)
wht_corr =fits.getdata('ica529030_pcorr_drz.fits',ext=2)

fig=plt.figure(figsize=(20,8))
ax1=fig.add_subplot(1,2,1)
ax2=fig.add_subplot(1,2,2)

ax1.imshow(wht,     vmin=0,vmax=800,cmap='Greys_r',origin='lower')
ax2.imshow(wht_corr,vmin=0,vmax=800,cmap='Greys_r',origin='lower')

ax1.set_title('ica529030_drz.fits (Pipeline WHT)',fontsize=20)
ax2.set_title('ica529030_pcorr_drz.fits (Corrected WHT)',fontsize=20)

ax1.plot([0,300,300,0,0],[250,250,350,350,250],c='red')
ax2.plot([0,300,300,0,0],[250,250,350,350,250],c='red')
ax1.plot([200,480,480,200,200],[360,360,460,460,360],c='red')
ax2.plot([200,480,480,200,200],[360,360,460,460,360],c='red')

## 4. Conclusions <a id="conclusions"></a>

Thank you for walking through this notebook. Now with WFC3 data, you should be familiar with:

- Examining the persistence models for a given dataset.
- Defining a threshold to use for masking pixels in the DQ array of FLT science frames.
- Reprocessing dithered frames with the new DQ flags to produce an improved combined DRZ image.

**Congratulations, you have completed the notebook.**

## Additional Resources <a id="add"></a>

Below are some additional resources that may be helpful. Please send any questions through the [HST Helpdesk](https://stsci.service-now.com/hst).

- [WFC3 Website](https://www.stsci.edu/hst/instrumentation/wfc3)
- [WFC3 Instrument Handbook](https://hst-docs.stsci.edu/wfc3ihb)
    - see Sections 5.7.9 and 7.9.4 for a discussion of IR persistence
- [WFC3 Data Handbook](https://hst-docs.stsci.edu/wfc3dhb)
    - see Section 8.1 for a discussion of IR persistence
    - see Section 8.2 for evaluating the amount of persistence in science images
    - see Section 8.3 for a discussion methods for mitigating persistence
- [DrizzlePac Handbook](https://hst-docs.stsci.edu/drizzpac)


## About this Notebook <a id="about"></a>

**Author:** Jennifer Mack, WFC3 Instrument Team

**Updated On:** 2021-09-28

## Citations <a id="cite"></a>
If you use `numpy`, `astropy`, `astroquery`, `wfc3tools`, or `drizzlepac` for published research, please cite the
authors. Follow these links for more information about citing the libraries below:

* [Citing `numpy`](https://numpy.org/citing-numpy/)
* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `astroquery`](https://astroquery.readthedocs.io/en/latest/license.html)
* [Citing `wfc3tools`](https://wfc3tools.readthedocs.io/en/latest/)
* [Citing `drizzlepac`](https://drizzlepac.readthedocs.io/en/latest/LICENSE.html)
***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 